# Computer Vision - Semester 8, Spring 2021

## Laboratory Project 1: Points of interest detection & feature extraction in images
    Christos Dimopoulos (031 17 037)
    Dimitris Dimos (031 17 165)

## Part 1: Edge detection in grayscale images

In [ ]:
# imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
from matplotlib import rcParams
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
# just to center image depiction 
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
## 1.1 Input images creation

# 1.1.1
Io_source = 'cv21_lab1_part1&2_material/edgetest_10.png'
Io = np.array(cv2.imread(Io_source, 0))
Io = Io.astype(np.float)/Io.max()   # normalize to [0, 1]
rcParams['figure.figsize'] = 4,4
plt.imshow(Io, cmap="gray")

#cv2.imshow('gray image', Io)
#cv2.waitKey(5000)
#cv2.destroyAllWindows()
#cv2.waitKey(1)

For the algorithms that follow, we will need our input images to be corrupted by **_White Gaussian Noise_** of zero mean and standard deviation $\sigma_{n}$ that depends on the desired PSNR.

By definition: $PSNR = 20\textrm{log}_{10} \bigg(\frac{I_{max} - I_{min}}{\sigma_n}\bigg) \,\,\,\, (dB)$,
where
$I_{max} = \underset{x,y}{\textrm{max}}I(x,y)$ and $I_{min} = \underset{x,y}{\textrm{min}}I(x,y)$

Therefore, $$ \sigma_{n} = (I_{max} - I_{min}) \cdot 10^{-\frac{PSNR}{20}} $$

In [ ]:
# 1.1.2
def find_sigma (PSNR: int, I: np.ndarray) -> float:
    Imax = np.amax(I)
    Imin = np.amin(I)
    sigma = (Imax - Imin) * 10**(-PSNR/20)
    return sigma

n_20 = np.random.normal(loc=0.0, scale=find_sigma(20, Io), size=Io.shape)
n_10 = np.random.normal(loc=0.0, scale=find_sigma(10, Io), size=Io.shape)

I_20 = Io + n_20
I_10 = Io + n_10

rcParams['figure.figsize'] = 9,9
fig, ax = plt.subplots(1,2)
ax[0].imshow(I_20, cmap="gray")
ax[1].imshow(I_10, cmap="gray")

The left image has been corrupted by noise with a PSNR of 20 dB and the right with a PSNR of 10 dB. It can easily be seen that the higher the PSNR, the higher the image clarity.  

In [ ]:
# auxiliary function to visualize convolution kernels in 3D
def plot3D_kernels (toPlot1: np.ndarray, toPlot2: np.ndarray) -> None:
    dim = toPlot1.shape[0]
    x = np.linspace(-(dim-1)/2, (dim-1)/2, dim)
    y = np.linspace(-(dim-1)/2, (dim-1)/2, dim)
    X, Y = np.meshgrid(x, y)
    
    fig = plt.figure(figsize=(25,10))
    
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    
    Z1 = np.array(toPlot1)
    ax.plot_wireframe(X, Y, Z1, color='black')#)cmap=cm.PuOr)
    plt.title('Gaussian Kernel', y = -0.01)
    
    ax = fig.add_subplot(1, 2, 2, projection='3d')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    
    Z2 = np.array(toPlot2)
    ax.plot_wireframe(X, Y, Z2, color='black')#)cmap=cm.PuOr)
    plt.title('Laplacian-of-Gaussian Kernel', y = -0.01)
    
    plt.show()

At this point, we will create the kernels of two linear filters:

$1.$&emsp; 2D Gaussian, using the openCV library: $ G_{\sigma}(x,y) = \frac{1}{2 \pi \sigma^2} e^{-\frac{x^2+y^2}{2\sigma^2}}$

$2.$&emsp; Laplacian of Gaussian (LoG), which we create manually: $ h(x,y) =  \nabla^2G_{\sigma}(x,y) $

To make it more clear, we will exploit the **separability** of the gaussian function and its derivatives to synthesize LoG.

&emsp;&emsp;$(a)$&emsp; We separate $G_{\sigma}(x,y)$ into 1D functions: 
$$
 G_{\sigma}(x,y) = \frac{1}{2 \pi \sigma^2} e^{-\frac{x^2+y^2}{2\sigma^2}} =
\frac{1}{2 \pi \sigma^2} \cdot
\underbrace{e^{-\frac{x^2}{2\sigma^2}}}_{G_{\sigma} (x)} \cdot
\underbrace{e^{-\frac{y^2}{2\sigma^2}}}_{G_{\sigma} (y)}
$$

&emsp;&emsp;$(b)$&emsp; The laplacian of $G_{\sigma}(x,y)$ is:
$$
\nabla^2G_{\sigma}(x,y) =
\frac{\partial^2 G_{\sigma}(x,y)}{\partial x^2} +
\frac{\partial^2 G_{\sigma}(x,y)}{\partial y^2} =
\frac{1}{2 \pi \sigma^2} \cdot
\bigg[
\frac{\partial^2 G_{\sigma}(x)}{\partial x^2}\cdot G_{\sigma}(y) +
\frac{\partial^2 G_{\sigma}(y)}{\partial y^2}\cdot G_{\sigma}(x)
\bigg]  \Longrightarrow \\[1cm]
\nabla^2 G_{\sigma}(x,y) = 
\frac{1}{2 \pi \sigma^2} \cdot
\bigg[
\frac{x^2 - \sigma^2}{\sigma^4} e^{-\frac{x^2}{2\sigma^2}} \cdot G_{\sigma}(y) +
\frac{y^2 - \sigma^2}{\sigma^4} e^{-\frac{y^2}{2\sigma^2}} \cdot G_{\sigma}(x)
\bigg] \Longrightarrow \\[1cm]
h(x,y) =
\frac{1}{2 \pi \sigma^2}
\bigg[
\underbrace{\frac{x^2 - \sigma^2}{\sigma^4} e^{-\frac{x^2}{2\sigma^2}}}_{newX} \cdot G_{\sigma}(y) +
\underbrace{\frac{y^2 - \sigma^2}{\sigma^4} e^{-\frac{y^2}{2\sigma^2}}}_{newY} \cdot G_{\sigma}(x)
\bigg]
$$

In [ ]:
## 1.2 Edge detection algorithms implementation

# 1.2.1
""" Produces 2D Gaussian filter response & its Laplacian (same spatial scale) """
def G_and_LoG (sigma: float) -> (np.ndarray,np.ndarray):
    kernelSize = int (np.ceil(3 * sigma)*2 + 1)
    
    # 2D Gaussian: G(x,y) - openCV library
    G = cv2.getGaussianKernel(kernelSize, sigma)
    G = G @ G.transpose()
    #normalize = sum(map(sum, G))
    #G = G/normalize 
    
    # Laplacian of Gaussian: (LoG) h(x,y) - manual construction
    coeff = 1/(2*np.pi*sigma**2)
    x = np.linspace(-(kernelSize-1)/2, (kernelSize-1)/2, kernelSize)
    y = np.linspace(-(kernelSize-1)/2, (kernelSize-1)/2, kernelSize)

    Gx = np.exp(-np.square(x)/(2*sigma**2))
    Gy = np.exp(-np.square(y)/(2*sigma**2))
    newX = (np.square(x)-sigma**2)/(sigma**4) * Gx
    newY = (np.square(y)-sigma**2)/(sigma**4) * Gy

    gridX, gridY       = np.meshgrid(Gx, Gy)
    gridNewX, gridNewY = np.meshgrid(newX, newY)
    
    LoG = coeff * ((gridY*gridNewX) + (gridX*gridNewY))
    
    return (G, LoG)

In [ ]:
# plotting an example of Gauss and LoG kernels with sigma = 100
Gauss, LoG = G_and_LoG(sigma=50)
plot3D_kernels(Gauss, LoG)

On the left side, we depict the 3D representation of a **Gaussian smoothing** (blurring) filter response and, on the right side, a **LoG** filter response.
***
At next, in order to approximate the **Laplacian of the smoothed image** we will use 2 techniques

&emsp;&emsp;$(i)$&emsp; linear convolution of the image with $h(x,y)$:
$L_1 = \nabla^2(G_{\sigma}*I) = (\nabla^2 G_{\sigma})*I = h*I $

&emsp;&emsp;$(ii)$&emsp; non-linear approximation using morphological filters:
$L_2 = I_{\sigma} \oplus B + I_{\sigma} \ominus B - 2 I_{\sigma} $

In [ ]:
# 1.2.2
def linearFilter (image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    output = cv2.filter2D(image, -1, kernel) # ddepth = -1 will produce output of the same depth as input
    return output

In [ ]:
# instance of linear filtering: for sigma = 1.2 and both PSNRs
sigma = 1.2
G, LoG = G_and_LoG(sigma)

Ismooth_20 = linearFilter(I_20, G)
L1_20 = linearFilter(I_20, LoG)

Ismooth_10 = linearFilter(I_10, G)
L1_10 = linearFilter(I_10, LoG)

rcParams['figure.figsize'] = 16,10
fig, ax = plt.subplots(2,3)

ax[0][0].imshow(I_20, cmap="gray")
ax[0][1].imshow(Ismooth_20, cmap="gray")
ax[0][2].imshow(L1_20, cmap="gray")

ax[1][0].imshow(I_10, cmap="gray")
ax[1][1].imshow(Ismooth_10, cmap="gray")
ax[1][2].imshow(L1_10, cmap="gray")

In the first column, we depict the corrupted images $I_{20 dB}$ and $I_{10 dB}$ without any filter application. The images of the second column are $I_{20 dB} * G$ and $I_{10 dB} * G$, respectively. The last column contains the Laplacian of smoothed image linear approximations $h*I$. For this experiment we have assumed that the standard deviation of both filter kernels is equal to $\sigma = 1.2$.

In [ ]:
def morpholFilter (Is: np.ndarray) -> np.ndarray:
    B = np.array([
    [0,1,0],
    [1,1,1],
    [0,1,0]
    ], dtype=np.uint8)
    
    dilated = cv2.dilate(Is, B)
    eroded = cv2.erode(Is, B)
    L2 = dilated + eroded - 2*Is
    return L2

In [ ]:
# instance of morphological filtering: for both PSNRs
L2_20 = morpholFilter(Ismooth_20)
L2_10 = morpholFilter(Ismooth_10)

rcParams['figure.figsize'] = 16,10
fig, ax = plt.subplots(2,3)

ax[0][0].imshow(I_20, cmap="gray")
ax[0][1].imshow(L1_20, cmap="gray")
ax[0][2].imshow(L2_20, cmap="gray")

ax[1][0].imshow(I_10, cmap="gray")
ax[1][1].imshow(L1_10, cmap="gray")
ax[1][2].imshow(L2_10, cmap="gray")

The images of the first column depict the corrupted $I_{20dB}$ and $I_{10dB}$ and in the second and third columns we see the outputs $L$ of linear and morphological (non-linear) filtering, respectively.
***
Our next step is to find the zerocrossings of the output $L$. For this cause, we will first produce the binary sign image $X$ of $L$ and then find the outline of $X$ by calculating:

$$ Y = (X \oplus B) - (X \ominus B) $$

In [ ]:
# 1.2.3
def zerocrossings (img: np.ndarray) -> np.ndarray:
    # X = (img >= 0)
    # X = X.astype(np.uint8)
    _, X = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY)
    B = np.array([
        [0,1,0],
        [1,1,1],
        [0,1,0]
        ], dtype=np.uint8)
    # Y = cv2.morphologyEx(X, cv2.MORPH_GRADIENT, B)
    dilated = cv2.dilate(X,B)
    eroded  = cv2.erode(X,B)
    Y = dilated - eroded
    return Y

In [ ]:
# instance of calculating zerocrossings: for both PSNRs
Y1_20 = zerocrossings(L1_20)
Y1_10 = zerocrossings(L1_10)

Y2_20 = zerocrossings(L2_20)
Y2_10 = zerocrossings(L2_10)

rcParams['figure.figsize'] = 13,13
fig, ax = plt.subplots(2,2)

ax[0][0].imshow(Y1_20, cmap="gray")
ax[0][1].imshow(Y2_20, cmap="gray")

ax[1][0].imshow(Y1_10, cmap="gray")
ax[1][1].imshow(Y2_10, cmap="gray")

The left images are the zerocrossings of the binary version of $L_1$ and the right images are the zerocrossings of the binary version of $L_2$.
***
We will now remove the zerocrossings that belong to relatively smooth regions.

In [ ]:
# 1.2.4
def disposeFakeEdges (Y: np.ndarray, Is: np.ndarray, theta: float) -> np.ndarray:
    gradIs = np.gradient(Is)
    ddy = gradIs[0]
    ddx = gradIs[1]
    
    normGradIs = np.sqrt( np.square(ddx) + np.square(ddy) )
    maxIs = np.amax(normGradIs)
    
    edges = np.where((Y == 1) & (normGradIs > theta*maxIs), 1, 0)
    return edges

In [ ]:
# instance of disposing relatively smooth zerocrossings: for both PSNRs
theta_20 = 0.17
theta_10 = 0.25

F1_20 = disposeFakeEdges(Y1_20, Ismooth_20, theta_20)
F1_10 = disposeFakeEdges(Y1_10, Ismooth_10, theta_10)

F2_20 = disposeFakeEdges(Y2_20, Ismooth_20, theta_20)
F2_10 = disposeFakeEdges(Y2_10, Ismooth_10, theta_10)

rcParams['figure.figsize'] = 13,13
fig, ax = plt.subplots(2,2)

ax[0][0].imshow(F1_20, cmap="gray_r")
ax[0][1].imshow(F2_20, cmap="gray_r")

ax[1][0].imshow(F1_10, cmap="gray_r")
ax[1][1].imshow(F2_10, cmap="gray_r")

The left images are the edges detected on $I_{20 dB}$ and the right images are the edges detected on $I_{10 dB}$, for $\theta_{egde} = 0.15$ and $0.20$, respectively
***
We will now construct an overall function to detect edges on images.

In [ ]:
'''
Usage: D = EdgeDetect(I, sigma, theta, linear)

Description:
    Returns a boolean matrix that represents 
    the edges of the input image

Input:
    I : input image
    sigma: standard deviation of the Gaussian distribution
    theta: threshold parameter
    LaplaceType: defines the model of filters that will be used in the algorithm    

        -- LaplaceType = 1 (Linear method - Convolution with LoG filter)
        -- LaplaceType = 0 (Non-Linear method - Convolution with morphological filter)

Output:
    D: a 2D boolean matrix containing the detected edges

'''
def EdgeDetect (img: np.ndarray, sigma: float, theta: float, linear: bool) -> np.ndarray:
    
    # Gaussian and Laplacian-of-Gaussian filters
    G, LoG = G_and_LoG(sigma)
    Is = linearFilter(img, G)
    
    if linear:
        L = linearFilter(img, LoG)
    else:
        L = morpholFilter(Is)
    
    D = zerocrossings(L)
    D = disposeFakeEdges(D, Is, theta)
    return D

We will now evaluate our algorithm's efficiency by calculating the following **quality criterion**:

$$ C = \frac{Pr(D|T) + Pr(T|D)}{2} $$

where:


&emsp;&emsp; $Pr(D|T)$ is the algorithm's **precision**: the percentage of the detected edges that are _actually real_

&emsp;&emsp; $Pr(T|D)$ is the algorithm's **recall**: the percentage of the real edges that were _detected_




In [ ]:
## 1.3 Evaluation of edge detection results

# 1.3.1
def RealEdges (img: np.ndarray) -> np.ndarray:
    B = np.array([
        [0,1,0],
        [1,1,1],
        [0,1,0]
        ], dtype=np.uint8)
    M = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, B)
    _, T = cv2.threshold(M, 0, 1, cv2.THRESH_BINARY)
    return T

In [ ]:
# getting the real edges for our evaluation
T = RealEdges(Io)
rcParams['figure.figsize'] = 4,4
plt.imshow(T,cmap='gray_r')
plt.title('Real edges')

The above picture represents the real edges of the input picture.

In [ ]:
def evaluate (D: np.ndarray, T: np.ndarray) -> float:
    all_detected = D.sum()
    all_real = T.sum()
    intersection_D_T = np.array((D == T) & (D == 1))
    intersection_D_T = intersection_D_T.astype(np.uint8)
    detected_that_are_real = intersection_D_T.sum()
    
    precision = detected_that_are_real/all_detected
    recall    = detected_that_are_real/all_real
    
    C = (precision + recall)/2
    return C

In [ ]:
# a typical experiment: sigma = 1.2, theta = 0.15
D20_linear  = EdgeDetect(img=I_20, sigma=1.2, theta=0.19, linear=1)
D20_morphol = EdgeDetect(img=I_20, sigma=1.2, theta=0.18, linear=0)

D10_linear  = EdgeDetect(img=I_10, sigma=2, theta=0.25, linear=1)
D10_morphol = EdgeDetect(img=I_10, sigma=2, theta=0.25, linear=0)

T = RealEdges(img=Io)

C20_linear  = round(evaluate(D20_linear, T)  * 100, 2)
C20_morphol = round(evaluate(D20_morphol, T) * 100, 2)
C10_linear  = round(evaluate(D10_linear, T)  * 100, 2)
C10_morphol = round(evaluate(D10_morphol, T) * 100, 2)

print('PSNR = 20 dB, Linear convolution with LoG: \x1b[32m{0} %\x1b[0m'.format(C20_linear))
print('')
print('PSNR = 20 dB, Morphological app/on of LoG: \x1b[32m{0} %\x1b[0m'.format(C20_morphol))
print('')
print('PSNR = 10 dB, Linear convolution with LoG: \x1b[31m{0} %\x1b[0m'.format(C10_linear))
print('')
print('PSNR = 10 dB, Morphological app/on of LoG: \x1b[31m{0} %\x1b[0m'.format(C10_morphol))

In [ ]:
# Experimental Evaluation using Linear Method
parameters_20dB = [(0.5,0.2), (0.5, 0.3), (1, 0,12), (1, 0.23), (1.2, 0.19), (1.2, 0.25), (1.5,0.2),
               (1.5, 0.25), (1.5, 0.3), (1.6, 0.18), (2, 0.1), (2, 0.18), (2, 0.25), (2, 0.25),
               (3, 0.15)]

parameters_10dB = [(1.2, 0.2), (1.2, 0.35), (1.2, 0.4), (2, 0.2), (2, 0.25), (2, 0.3), (2, 0.4), (2.5, 0.15),
               (2.5, 0.25), (2.5, 0.3), (3, 0.18), (3, 0.1), (4, 0.18), (4, 0.25), (4, 0.25),
               (1, 0.15)]

def compare_parameters(image, parameters, T, decibel = '20 dB', linear_mode = 1):
    experiment = []
    for i in range(len(parameters)):
        experiment.append(EdgeDetect(img=image, sigma = parameters[i][0], 
                                     theta=parameters[i][1], linear = linear_mode))
    for i in range(len(parameters)):
        print(decibel, " Noisy Image - Sigma = ", parameters[i][0], 
              " Theta = ", parameters[i][1],": C = ", round(evaluate(experiment[i],T),4))


print ('------------- Edge Detection with Linear Method -------------\n')

compare_parameters(I_20, parameters_20dB, T, decibel = '20 dB', linear_mode = 1)
print(" ")
compare_parameters(I_10, parameters_10dB, T, decibel = '10 dB', linear_mode = 1)

In [ ]:
# Experimental Evaluation using Non - Linear Method
print ('------------- Edge Detection with Morphological Filters -------------\n')

compare_parameters(I_20, parameters_20dB, T, decibel = '20 dB', linear_mode = 0)
print(" ")
compare_parameters(I_10, parameters_10dB, T, decibel = '10 dB', linear_mode = 0)

In [ ]:
## 1.4 Application of Edge Detection algorithm on real images

# 1.4.1
img_source = 'cv21_lab1_part1&2_material/urban_edges.jpg'
img = np.array(cv2.imread(img_source, 1))
RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img = gray_img.astype(np.float)/gray_img.max()   # normalize to [0, 1]

rcParams['figure.figsize'] = 15,15
fig, ax = plt.subplots(1,2)

ax[0].imshow(RGB_img)
ax[1].imshow(gray_img, cmap='gray')

ax[0].set_axis_off()
ax[1].set_axis_off()

ax[0].title.set_text('Original Image')
ax[1].title.set_text('Grayscale Image')

In [ ]:
# Linear Edge Detection
edges_linear = EdgeDetect(gray_img, sigma=1.2, theta=0.15, linear=1)
# Non Linear Edge Detection
edges_morph = EdgeDetect(gray_img, sigma=1.2, theta=0.15, linear=0)

rcParams['figure.figsize'] = 20,15
fig, ax = plt.subplots(1,2)

ax[0].imshow(edges_linear, cmap='gray')
ax[1].imshow(edges_morph, cmap='gray')

ax[0].set_axis_off()
ax[1].set_axis_off()

ax[0].title.set_text('Linear Method')
ax[1].title.set_text('Morphological Method')


At first, the **color** input picture is converted to **grayscale** and, then, inserted into our _EdgeDetect()_ algorithm to produce the right-most picture.

In [ ]:
# 1.4.2
# Experimenting on different parameter values
# Experimental Evaluation using Non - Linear Method
sigma_vector = [0.1, 0.1, 0.2, 1, 1.5, 2]
theta_vector = [0.15, 0.3, 0.2, 0.2, 0.3, 0.2]

experiment = []
for i in range(len(sigma_vector)):
    experiment.append(EdgeDetect(img=gray_img, sigma = sigma_vector[i], theta=theta_vector[i], linear = False))

In [ ]:
# Experimenting on theta parameter for low sigma values
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots(1,3)

ax[0].imshow(experiment[0], cmap='gray')
ax[1].imshow(experiment[1], cmap='gray')
ax[2].imshow(experiment[2], cmap='gray')

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

ax[0].title.set_text('σ = 0.1, θ = 0.15')
ax[1].title.set_text('σ = 0.1, θ = 0.3')
ax[2].title.set_text('σ = 0.2, θ = 0.15')

In [ ]:
# Experimenting on larger sigma values
rcParams['figure.figsize'] = 20,20
fig, ax = plt.subplots(1,3)

ax[0].imshow(experiment[3], cmap='gray')
ax[1].imshow(experiment[4], cmap='gray')
ax[2].imshow(experiment[5], cmap='gray')

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

ax[0].title.set_text('σ = 1, θ = 0.2')
ax[1].title.set_text('σ = 1.5, θ = 0.3')
ax[2].title.set_text('σ = 2, θ = 0.2')

In [ ]:
# Best Result
rcParams['figure.figsize'] = 10,10
fig, ax = plt.subplots(1,1)

ax.imshow(experiment[2], cmap='gray')
ax.set_axis_off()